# OpenAI Swarm: A Framework for Multi-Agent Orchestration 

In [ ]:
!pip install git+https://github.com/openai/swarm.git

In [1]:
import logging

# Configure logging to display in the notebook
logger = logging.getLogger(__name__)
logger.setLevel(logging.WARNING)

# Create a stream handler to output logs to the notebook
stream_handler = logging.StreamHandler()
stream_handler.setLevel(logging.INFO)

# Add a simple format to the handler
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
stream_handler.setFormatter(formatter)

# Add the handler to the logger
logger.addHandler(stream_handler)

## Arcade-AI

In [2]:
from arcadepy import Arcade
import os
from dotenv import load_dotenv

load_dotenv()

arcade_client = Arcade()

In [3]:
from typing import List, Dict, Any
from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain_arcade._utilities import tool_definition_to_pydantic_model

# Define a mapping from JSON types to Python types
json_to_python_types = {
    "string": "str",
    "integer": "int",
    "number": "float",
    "boolean": "bool",
    "array": "list",
    "object": "dict",
}

def map_json_type(param_details):
    """Map JSON schema type to Python type, handling nested structures."""
    json_type = param_details.get("type")
    
    if json_type == "array":
        # If it's an array, map the type of the items recursively
        item_details = param_details.get("items", {})
        item_type = map_json_type(item_details)
        return f"List[{item_type}]"
    elif json_type == "object":
        # If it's an object, we assume it represents a dictionary with string keys
        properties = param_details.get("properties", {})
        # Map each property's type within the dictionary
        mapped_properties = {k: map_json_type(v) for k, v in properties.items()}
        # Represent as Dict with specific types or Any if properties are complex
        return f"Dict[str, {', '.join(mapped_properties.values())}]" if mapped_properties else "Dict[str, Any]"
    else:
        # Map primitive types directly
        return json_to_python_types.get(json_type, "Any")  # Default to `Any` for unrecognized types

def create_dynamic_function(json_schema):

    # Extract the function details from the schema
    function_details = json_schema.get("function", {})
    function_name = function_details.get("name")
    description = function_details.get("description", "")
    parameters = function_details.get("parameters", {}).get("properties", {})
    required_params = function_details.get("parameters", {}).get("required", [])

    # Build the function signature
    param_list = []
    for param_name, param_details in parameters.items():
        python_type = map_json_type(param_details)
        if param_name in required_params:
            param_list.append(f"{param_name}: {python_type}")
        else:
            default_value = "None" if "List" in python_type or "Dict" in python_type or python_type == "str" else "0"
            param_list.append(f"{param_name}: {python_type} = {default_value}")
    
    param_str = ", ".join(param_list)

    # Build the docstring with parameter descriptions
    docstring_params = []
    for param_name, param_details in parameters.items():
        required = "Required. " if param_name in required_params else ""
        docstring_params.append(f"{param_name} ({map_json_type(param_details)}): {required}{param_details.get('description', '')}")
    
    params_str = "\n".join(docstring_params)

    # Create the function code
    func_code = f'''
def {function_name}({param_str}):
    """{description}

    Args:
        {params_str}
    """
    kwargs = locals()
    logger.info(kwargs)
    # Filter out None values from kwargs
    filtered_kwargs = {{k: v for k, v in kwargs.items() if v is not None}}
    response = arcade_client.tools.execute(
        tool_name="{function_name}",
        inputs=filtered_kwargs,
        user_id=os.environ['ARCADE_USER_ID']
    )
    logger.info(response)
    return response
'''

    # Define the function dynamically
    local_scope = {
        "arcade_client": arcade_client,  # Add required dependencies
        "os": os,
        "logger": logger,
        "List": List,
        "Dict": Dict,
        "Any": Any
    }
    exec(func_code, globals(), local_scope)
    return local_scope[function_name]

def get_tool(tool_id):
    schema = convert_to_openai_tool(tool_definition_to_pydantic_model(arcade_client.tools.get(tool_id=tool_id)))
    schema['function']['name'] = tool_id
    tool = create_dynamic_function(schema)
    logger.info(schema)
    return tool

tool = get_tool("Google_CreateEvent")

## Main Agent using Tools

In [4]:
from swarm import Agent, Swarm
from typing import List, Callable

AGENT_SYSTEM_PROMPT = """
# Task
Tu nombre es Gabriela y sos un asistente de IA diseñado para ayudar al equipo de Pampa Labs. 
                      
# Guidelines                   
Tus respuestas deben ser:

1. Amigables y accesibles, usando un tono cálido
2. Concisas y al grano, evitando verbosidad innecesaria
3. Útiles e informativas, proporcionando información precisa
4. Respetuosas de la privacidad del usuario y los límites éticos

Solo puedes ayudar usando las herramientas disponibles y con pedidos que vengan de miembros del equipo. Todo lo que no se pueda responder usando las herramientas, debes decir que no puedes ayudar y disculparte.
"""

# I had to convert Pydantic models to function arguments with corresponding docstrings
# This involves extracting fields from the Pydantic models and creating function
# parameters with appropriate type hints and descriptions in the docstring.
def set_meal_tool(context_variables, meal: str, date: str):
    """
    Sets the meal plan for a specific date.

    Args:
        meal (str): The name of the meal.
        date (str): The date of the meal plan.
    """
    return f"Meal plan created and sent to the provider: {meal} for {date} by team member {context_variables['id']}"

def get_expenses_tool(context_variables):
    """
    Retrieves all pending expenses
    """

    # Mock implementation for testing purposes
    mock_expenses = {
        "user1": {
            'expenses': [
                {
                "expense_type": "food",
                "date": "2023-05-01",
                "total_value": 50.0,
                "state": "pending"
            },
            {
                "expense_type": "coffee",
                "date": "2023-05-02",
                "total_value": 5.0,
                "state": "pending"
                }
            ]
        }
    }
    # Functions called by an Agent should return a type that has __str__
    return f"Expenses retrieved: {mock_expenses[context_variables['id']]['expenses']}"

class GabyAgent:
    def __init__(self,
        prompt = AGENT_SYSTEM_PROMPT,
        model_name: str = "gpt-4o",
        tools: List[Callable] = [],
    ):

        self.agent = Agent(
            name="Main Agent",
            instructions=prompt,
            functions=tools,
            model=model_name
        )
        self.client = Swarm()

    def invoke(self, id, messages, debug=False):
        response = self.client.run(
            agent=self.agent,
            messages=messages,
            debug=debug,
            context_variables={"id": id}
        )
        return response.messages[-1]["content"]
    
    def stream(self, id, messages, debug=False):
        stream = self.client.run(
            agent=self.agent,
            messages=messages,
            debug=debug,
            context_variables={"id": id},
            stream=True
        )
        for chunk in stream:
            yield chunk

In [5]:
gaby_agent = GabyAgent(tools=[set_meal_tool, get_expenses_tool, tool])
response_content = gaby_agent.invoke(
    id="user1",
    messages=[
        {"role": "user", "content": "I want to set the meal plan for the date 2024-05-01. The meal is lasagna."},
        {"role": "user", "content": "I want to get the expenses"},
        {"role": "user", "content": "Calendar event for the date 2024-10-31 at 10:00 PM - 11:00 PM argentina timezone. The event is a meeting with Halloween party."}
    ],
    debug=False
)
print(response_content)

¡Todo listo! 

1. **Comida Programada**: El plan de comida para el 1 de mayo de 2024 es lasagna.

2. **Gastos Pendientes**:
   - Tipo: Comida, Fecha: 2023-05-01, Total: $50.00
   - Tipo: Café, Fecha: 2023-05-02, Total: $5.00

3. **Evento del Calendario**: El evento "Reunión con Fiesta de Halloween" se ha programado exitosamente para el 31 de octubre de 2024, de 10:00 PM a 11:00 PM en la zona horaria de Argentina. Puedes ver los detalles del evento [aquí](https://www.google.com/calendar/event?eid=YjhmaG0yMDAyNzViYzQyOWppa2tsODNqZzggcGFtcGFsZWFybkBt).

Si necesitas algo más, aquí estoy para ayudar! 😊
